# Big Data Coursework 2016/7 
# Part 1: Spam Detection with Spark
## Muaaz Bin Sarfaraz & Chadi-El-Hajj


In [1]:
# Make RDD from all text files in directory
filePath = 'hdfs://saltdean.nsqdc.city.ac.uk/data/spam/bare/part1' 
textFiles_RDD = sc.wholeTextFiles(filePath)
textFiles_RDD.top(3)

[('hdfs://saltdean.nsqdc.city.ac.uk/data/spam/bare/part1/spmsga141.txt',
  "Subject: advertisement\n\nthis message complies with the proposed united states federal requirements for commercial email . for additional information see : your e - mail address is on a list saying you would like to information on this type of service . further mailings to you may be stopped at no cost to you by sending a reply with remove as the subject . all remove requests will be honored immediately as outlined under the murkowski bill at . attempts to cancel any aforementioned email accounts will result in the inability to process remove requests , your or anyone else 's . please include all addresses where you receive email to avoid receiving further mailings from us . ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ are you haveing a hard time finding places that will give you a credit card or a merchant card . if

In [48]:
# Split the text into words (lower case), creating a (file,word) RDD. 
import re
def splitFileWords(filenameContent): # your splitting function
    f,w = filenameContent # split the input tuple  
    fwLst = [] # the new list for (filename,word) tuples
    wLst = re.split('\W+',w) # <<< now create a word list wLst
    for w in wLst: # iterate through the list
        fwLst.append((f,w)) # <<< and append (f,w) to the 
    return fwLst #return a list of (f,w) tuples 
    
fw_RDD = textFiles_RDD.flatMap(splitFileWords) #flatmap used to flatten all files
#fwlower= fw_RDD.map(x.lower()) #all words mapped in lower case
fw_RDD.top(3)



[('hdfs://saltdean.nsqdc.city.ac.uk/data/spam/bare/part1/spmsga141.txt',
  'your'),
 ('hdfs://saltdean.nsqdc.city.ac.uk/data/spam/bare/part1/spmsga141.txt',
  'your'),
 ('hdfs://saltdean.nsqdc.city.ac.uk/data/spam/bare/part1/spmsga141.txt',
  'your')]

In [51]:
#Use the code from the labs to generate the [(word,count), ...] list per file and
#to create a word frequency vector. Normalise the term frequency (TF) vector by the
#total word count.
from operator import add
fw_1_RDD = fw_RDD.map(lambda x: (x.lower(),1))  #<<< change (f,w) to ((f,w),1)
fw_c_RDD = fw_1_RDD.reduceByKey(add) #<<< count the words
print(fw_c_RDD.top(3))


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 56.0 failed 4 times, most recent failure: Lost task 1.3 in stage 56.0 (TID 173, 10.207.1.107): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 317, in func
    return f(iterator)
  File "/usr/local/spark/python/pysp

In [33]:
def reGrpLst(fwc): # we get a nested tuple
    fw,c = fwc # unpack the tuple into the (filename,word) and the count
    f,w = fw # unpack the (filename,word) into filename and word 
    return (f,[(w,c)]) # return (f,[(w,c)]) structure. Can be used verbatim, if your variable names match.

f_wcL_RDD = fw_c_RDD.map(reGrpLst) 
f_wcL_RDD.take(3)

[('hdfs://saltdean.nsqdc.city.ac.uk/data/spam/bare/part1/5-1230msg1.txt',
  [('quite', 1)]),
 ('hdfs://saltdean.nsqdc.city.ac.uk/data/spam/bare/part1/spmsga140.txt',
  [('that', 31)]),
 ('hdfs://saltdean.nsqdc.city.ac.uk/data/spam/bare/part1/3-419msg1.txt',
  [('tone', 1)])]

In [41]:
count = 0
def normalise(fwc): # we get a nested tuple
    if f=f[0]
    count=c+count
    return (f,[(w,c)]) # return (f,[(w,c)]) structure. Can be used verbatim, if your variable names match.

fwN_RDD=f_wcL_RDD.map(normalise) #normalise term frequency by total word count
fwN_RDD.top(3)
#accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / test.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 41.0 failed 4 times, most recent failure: Lost task 0.3 in stage 41.0 (TID 128, 10.207.1.106): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 797, in func
    initial = next(iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 1233, in topIterator
    yield heapq.nlargest(num, iterator, key=key)
  File "/usr/li

## Hashing 

 

In [ ]:
def hashing_vectorizer(word_count_list, N):
     v = [0] * N  # create fixed size vector of 0s
     for word_count in word_count_list: 
         word,count = word_count # unpack tuple
         h = hash(word) # get hash value
         v[h % N] = v[h % N] + count # add count
     return v 
N=100
fwH_RDD= fwN_RDD.map(hashing_vectorizer)
fwH_RDD.top(3)
#print(f_wVec_RDD.top(3))

## NaiveBayes Model
Detecting spam and ham

In [ ]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
# RDD with labelpoint

def label(fwc): # we get a nested tuple
    f,wc = fwc # unpack the tuple into the (filename,word) and the count
        if "spmsg" in f:
            return (1,[(w,c)])
        else:
            return (0,[(w,c)])
data=fwH_RDD.map(label)
# Train a naive Bayes model.
training, test = data.randomSplit([0.6, 0.4])
model = NaiveBayes.train(training, 1.0)
# Make prediction and test accuracy.
predictionAndLabel = test.map(lambda p: (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / test.count()
print('model accuracy {}'.format(accuracy))

# Save and load model
output_dir = 'target/tmp/myNaiveBayesModel'
shutil.rmtree(output_dir, ignore_errors=True)
model.save(sc, output_dir)
sameModel = NaiveBayesModel.load(sc, output_dir)
predictionAndLabel = test.map(lambda p: (sameModel.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / test.count()
print('sameModel accuracy {}'.format(accuracy))